<a href="https://colab.research.google.com/github/ecfairle/trends/blob/main/trends.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Reddit AI Keyword Trend Analysis
# This notebook analyzes keyword trends from AI-focused subreddits over the past 6 months

import praw
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from collections import Counter, defaultdict
import re
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import time
import warnings
warnings.filterwarnings('ignore')

# Download required NLTK data
try:
    nltk.download('punkt', quiet=True)
    nltk.download('stopwords', quiet=True)
except:
    print("NLTK download failed - continuing without it")

# Reddit API Configuration
# Create a Reddit app at https://www.reddit.com/prefs/apps
REDDIT_CLIENT_ID = "your_client_id_here"
REDDIT_CLIENT_SECRET = "your_client_secret_here"
REDDIT_USER_AGENT = "AITrendAnalysis/1.0 by YourUsername"  # Replace with your username

class RedditAIAnalyzer:
    def __init__(self, client_id, client_secret, user_agent):
        """Initialize Reddit API client"""
        self.reddit = praw.Reddit(
            client_id=client_id,
            client_secret=client_secret,
            user_agent=user_agent
        )

        # AI-focused subreddits to analyze
        self.ai_subreddits = [
            'MachineLearning',      # 2.8M members - academic/research focused
            'artificial',           # 200K members - general AI discussion
            'singularity',          # 400K members - AI future/AGI discussion
            'ChatGPT',             # 300K members - ChatGPT specific
            'OpenAI',              # 100K members - OpenAI developments
            'LocalLLaMA',          # 150K members - local AI models
            'ArtificialIntelligence', # 180K members - general AI
            'deeplearning',        # 120K members - deep learning focus
            'compsci',             # 500K members - computer science (AI subset)
            'MachineLearningNews'  # 50K members - AI news and updates
        ]

        # Extended stopwords for cleaner analysis
        try:
            self.stop_words = set(stopwords.words('english'))
        except:
            self.stop_words = {'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'of', 'with', 'by', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'do', 'does', 'did', 'will', 'would', 'could', 'should', 'may', 'might', 'must', 'can', 'this', 'that', 'these', 'those', 'i', 'you', 'he', 'she', 'it', 'we', 'they', 'me', 'him', 'her', 'us', 'them'}

        self.stop_words.update([
            'reddit', 'post', 'comment', 'thread', 'subreddit', 'upvote', 'downvote',
            'https', 'http', 'www', 'com', 'co', 'amp', 'via', 'edit', 'update',
            'today', 'new', 'great', 'good', 'best', 'check', 'get', 'make',
            'see', 'like', 'know', 'think', 'work', 'time', 'way', 'use', 'using',
            'people', 'lot', 'really', 'much', 'would', 'also', 'even', 'still',
            'im', 'ive', 'dont', 'cant', 'wont', 'isnt', 'arent', 'wasnt', 'werent'
        ])

        self.request_count = 0

    def log_request(self):
        """Track API requests"""
        self.request_count += 1
        if self.request_count % 10 == 0:
            print(f"   Made {self.request_count} API requests so far...")

    def get_subreddit_posts(self, subreddit_name, time_filter='month', limit=100):
        """Get posts from a subreddit within a time period"""
        try:
            subreddit = self.reddit.subreddit(subreddit_name)
            self.log_request()

            posts_data = []

            # Get top posts from the time period
            for post in subreddit.top(time_filter=time_filter, limit=limit):
                self.log_request()

                # Skip stickied/pinned posts
                if post.stickied:
                    continue

                post_data = {
                    'id': post.id,
                    'title': post.title,
                    'selftext': post.selftext if hasattr(post, 'selftext') else '',
                    'score': post.score,
                    'num_comments': post.num_comments,
                    'created_utc': datetime.fromtimestamp(post.created_utc),
                    'url': post.url,
                    'subreddit': subreddit_name,
                    'author': str(post.author) if post.author else '[deleted]'
                }

                # Get top comments for additional context
                post.comments.replace_more(limit=0)  # Remove "load more comments"
                comments = []
                for comment in post.comments.list()[:10]:  # Top 10 comments
                    if hasattr(comment, 'body') and len(comment.body) > 20:
                        comments.append(comment.body)

                post_data['top_comments'] = comments
                posts_data.append(post_data)

                # Small delay to be respectful to Reddit's servers
                if len(posts_data) % 20 == 0:
                    time.sleep(1)

            return posts_data

        except Exception as e:
            print(f"Error fetching posts from r/{subreddit_name}: {e}")
            return []

    def filter_posts_by_date(self, posts, start_date, end_date):
        """Filter posts by date range"""
        filtered = []
        for post in posts:
            if start_date <= post['created_utc'] <= end_date:
                filtered.append(post)
        return filtered

    def extract_text_content(self, posts):
        """Extract all text content from posts and comments"""
        all_text = []

        for post in posts:
            # Combine title and selftext
            text_parts = [post['title']]
            if post['selftext'] and len(post['selftext']) > 10:
                text_parts.append(post['selftext'])

            # Add top comments
            text_parts.extend(post['top_comments'])

            # Join all text for this post
            full_text = ' '.join(text_parts)
            all_text.append(full_text)

        return all_text

    def clean_text(self, text):
        """Clean and preprocess text"""
        # Remove URLs
        text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
        # Remove Reddit-specific formatting
        text = re.sub(r'/r/\w+|/u/\w+|u/\w+|r/\w+', '', text)
        # Remove special characters but keep spaces
        text = re.sub(r'[^\w\s]', ' ', text)
        # Convert to lowercase
        text = text.lower()
        # Remove extra whitespace
        text = re.sub(r'\s+', ' ', text).strip()

        return text

    def extract_keywords(self, texts, min_freq=3, max_features=200):
        """Extract keywords using TF-IDF"""
        if not texts:
            return []

        # Clean all texts
        cleaned_texts = [self.clean_text(text) for text in texts if text.strip()]

        if not cleaned_texts:
            return []

        try:
            # Use TF-IDF to extract important terms
            vectorizer = TfidfVectorizer(
                max_features=max_features,
                ngram_range=(1, 3),  # Include up to 3-word phrases
                min_df=min_freq,
                max_df=0.8,  # Ignore terms that appear in >80% of documents
                stop_words=list(self.stop_words),
                token_pattern=r'\b[a-zA-Z]{3,}\b'  # Only words with 3+ letters
            )

            tfidf_matrix = vectorizer.fit_transform(cleaned_texts)
            feature_names = vectorizer.get_feature_names_out()
            scores = tfidf_matrix.sum(axis=0).A1

            # Create keyword-score pairs
            keyword_scores = list(zip(feature_names, scores))
            keyword_scores.sort(key=lambda x: x[1], reverse=True)

            return keyword_scores

        except Exception as e:
            print(f"Error in keyword extraction: {e}")
            return []

    def analyze_trends(self, months_back=6, posts_per_subreddit=100):
        """Analyze keyword trends across AI subreddits"""
        print(f"Analyzing AI keyword trends across {len(self.ai_subreddits)} subreddits")
        print(f"Time period: {months_back} months back")
        print("="*60)

        # Define time periods
        end_date = datetime.now()
        mid_date = end_date - timedelta(days=months_back*30//2)  # 3 months ago
        start_date = end_date - timedelta(days=months_back*30)   # 6 months ago

        print(f"Period 1 (Earlier): {start_date.date()} to {mid_date.date()}")
        print(f"Period 2 (Recent):  {mid_date.date()} to {end_date.date()}")
        print()

        all_period1_posts = []
        all_period2_posts = []
        subreddit_stats = {}

        for i, subreddit_name in enumerate(self.ai_subreddits):
            print(f"Processing r/{subreddit_name} ({i+1}/{len(self.ai_subreddits)})...")

            # Get posts from the full 6-month period
            try:
                # For period 1, we need posts from 6 months ago
                posts_6m = self.get_subreddit_posts(subreddit_name, 'all', limit=posts_per_subreddit*2)

                # Filter posts by date periods
                period1_posts = self.filter_posts_by_date(posts_6m, start_date, mid_date)
                period2_posts = self.filter_posts_by_date(posts_6m, mid_date, end_date)

                # If we don't have enough older posts, get more recent ones
                if len(period1_posts) < 20:
                    recent_posts = self.get_subreddit_posts(subreddit_name, 'month', limit=posts_per_subreddit)
                    period2_posts.extend(recent_posts[:posts_per_subreddit//2])

                all_period1_posts.extend(period1_posts)
                all_period2_posts.extend(period2_posts)

                subreddit_stats[subreddit_name] = {
                    'period1_posts': len(period1_posts),
                    'period2_posts': len(period2_posts),
                    'total_posts': len(period1_posts) + len(period2_posts)
                }

                print(f"   Found {len(period1_posts)} posts (period 1), {len(period2_posts)} posts (period 2)")

            except Exception as e:
                print(f"   Error processing r/{subreddit_name}: {e}")
                subreddit_stats[subreddit_name] = {'error': str(e)}

            # Small delay between subreddits
            time.sleep(1)

        print(f"\nTotal posts collected:")
        print(f"Period 1: {len(all_period1_posts)} posts")
        print(f"Period 2: {len(all_period2_posts)} posts")
        print(f"Total API requests made: {self.request_count}")

        # Extract text content
        print("\nExtracting text content...")
        period1_texts = self.extract_text_content(all_period1_posts)
        period2_texts = self.extract_text_content(all_period2_posts)

        # Extract keywords
        print("Extracting keywords for period 1...")
        keywords1 = self.extract_keywords(period1_texts, min_freq=2)

        print("Extracting keywords for period 2...")
        keywords2 = self.extract_keywords(period2_texts, min_freq=2)

        return {
            'period1_keywords': dict(keywords1),
            'period2_keywords': dict(keywords2),
            'period1_posts': all_period1_posts,
            'period2_posts': all_period2_posts,
            'period1_texts': period1_texts,
            'period2_texts': period2_texts,
            'subreddit_stats': subreddit_stats,
            'date_ranges': {
                'period1': f"{start_date.date()} to {mid_date.date()}",
                'period2': f"{mid_date.date()} to {end_date.date()}"
            },
            'api_requests': self.request_count
        }

    def create_visualizations(self, trend_data):
        """Create comprehensive visualizations of keyword trends"""
        period1_keywords = trend_data['period1_keywords']
        period2_keywords = trend_data['period2_keywords']

        # Create comparison dataset
        all_keywords = set(period1_keywords.keys()) | set(period2_keywords.keys())

        comparison_data = []
        for keyword in all_keywords:
            score1 = period1_keywords.get(keyword, 0)
            score2 = period2_keywords.get(keyword, 0)
            difference = score2 - score1

            # Calculate percentage change (avoid division by zero)
            if score1 > 0:
                percent_change = ((score2 - score1) / score1) * 100
            elif score2 > 0:
                percent_change = 200  # New keyword
            else:
                percent_change = 0

            comparison_data.append({
                'keyword': keyword,
                'period1_score': score1,
                'period2_score': score2,
                'difference': difference,
                'percent_change': percent_change,
                'avg_score': (score1 + score2) / 2
            })

        df = pd.DataFrame(comparison_data)
        df = df.sort_values('difference', key=abs, ascending=False)

        # Create visualizations
        fig = plt.figure(figsize=(20, 24))

        # 1. Top growing keywords
        plt.subplot(3, 2, 1)
        top_growing = df.nlargest(20, 'difference')
        colors = ['green' if x > 0 else 'red' for x in top_growing['difference']]
        plt.barh(range(len(top_growing)), top_growing['difference'], color=colors, alpha=0.7)
        plt.yticks(range(len(top_growing)), top_growing['keyword'])
        plt.title('Top 20 Growing Keywords (Recent vs Earlier Period)', fontsize=14, fontweight='bold')
        plt.xlabel('TF-IDF Score Difference')
        plt.grid(axis='x', alpha=0.3)
        plt.gca().invert_yaxis()

        # 2. Top declining keywords
        plt.subplot(3, 2, 2)
        top_declining = df.nsmallest(20, 'difference')
        colors = ['red' if x < 0 else 'green' for x in top_declining['difference']]
        plt.barh(range(len(top_declining)), top_declining['difference'], color=colors, alpha=0.7)
        plt.yticks(range(len(top_declining)), top_declining['keyword'])
        plt.title('Top 20 Declining Keywords (Recent vs Earlier Period)', fontsize=14, fontweight='bold')
        plt.xlabel('TF-IDF Score Difference')
        plt.grid(axis='x', alpha=0.3)
        plt.gca().invert_yaxis()

        # 3. Scatter plot: Period 1 vs Period 2 scores
        plt.subplot(3, 2, 3)
        plt.scatter(df['period1_score'], df['period2_score'], alpha=0.6, s=60)
        max_score = max(df[['period1_score', 'period2_score']].max())
        plt.plot([0, max_score], [0, max_score], 'r--', alpha=0.5, label='Equal usage line')
        plt.xlabel('Earlier Period TF-IDF Score')
        plt.ylabel('Recent Period TF-IDF Score')
        plt.title('Keyword Usage Comparison: Earlier vs Recent Period')
        plt.legend()
        plt.grid(alpha=0.3)

        # Annotate notable keywords
        for _, row in df.head(15).iterrows():
            if abs(row['difference']) > df['difference'].std() * 1.5:
                plt.annotate(row['keyword'][:15],
                           (row['period1_score'], row['period2_score']),
                           xytext=(5, 5), textcoords='offset points',
                           fontsize=8, alpha=0.8,
                           bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.3))

        # 4. Percentage change distribution
        plt.subplot(3, 2, 4)
        # Filter extreme outliers for better visualization
        filtered_changes = df[abs(df['percent_change']) < 500]['percent_change']
        plt.hist(filtered_changes, bins=50, alpha=0.7, color='skyblue', edgecolor='black')
        plt.axvline(x=0, color='red', linestyle='--', alpha=0.7, label='No change')
        plt.xlabel('Percentage Change (%)')
        plt.ylabel('Number of Keywords')
        plt.title('Distribution of Keyword Usage Changes')
        plt.legend()
        plt.grid(alpha=0.3)

        # 5. Word cloud for Period 1
        plt.subplot(3, 2, 5)
        if trend_data['period1_texts']:
            text1 = ' '.join([self.clean_text(text) for text in trend_data['period1_texts']])
            # Remove common words for better visualization
            for word in ['ai', 'model', 'data', 'learning', 'machine', 'use', 'work', 'new', 'get']:
                text1 = re.sub(rf'\b{word}\b', '', text1)

            try:
                wordcloud1 = WordCloud(
                    width=800, height=400,
                    background_color='white',
                    stopwords=self.stop_words,
                    max_words=100,
                    colormap='viridis'
                ).generate(text1)
                plt.imshow(wordcloud1, interpolation='bilinear')
                plt.title(f'Word Cloud - {trend_data["date_ranges"]["period1"]}', fontsize=12)
                plt.axis('off')
            except:
                plt.text(0.5, 0.5, 'Word Cloud\nGeneration Failed', ha='center', va='center', transform=plt.gca().transAxes)
                plt.title(f'Period 1: {trend_data["date_ranges"]["period1"]}')

        # 6. Word cloud for Period 2
        plt.subplot(3, 2, 6)
        if trend_data['period2_texts']:
            text2 = ' '.join([self.clean_text(text) for text in trend_data['period2_texts']])
            # Remove common words for better visualization
            for word in ['ai', 'model', 'data', 'learning', 'machine', 'use', 'work', 'new', 'get']:
                text2 = re.sub(rf'\b{word}\b', '', text2)

            try:
                wordcloud2 = WordCloud(
                    width=800, height=400,
                    background_color='white',
                    stopwords=self.stop_words,
                    max_words=100,
                    colormap='plasma'
                ).generate(text2)
                plt.imshow(wordcloud2, interpolation='bilinear')
                plt.title(f'Word Cloud - {trend_data["date_ranges"]["period2"]}', fontsize=12)
                plt.axis('off')
            except:
                plt.text(0.5, 0.5, 'Word Cloud\nGeneration Failed', ha='center', va='center', transform=plt.gca().transAxes)
                plt.title(f'Period 2: {trend_data["date_ranges"]["period2"]}')

        plt.suptitle('Reddit AI Keyword Trends Analysis', fontsize=16, fontweight='bold', y=0.98)
        plt.tight_layout()
        plt.show()

        # Print detailed results
        print("\n" + "="*80)
        print("🔍 REDDIT AI KEYWORD TRENDS ANALYSIS RESULTS")
        print("="*80)

        print(f"\n📊 DATA COLLECTION SUMMARY:")
        print(f"   • Total posts analyzed: {len(trend_data['period1_posts']) + len(trend_data['period2_posts'])}")
        print(f"   • Period 1 posts: {len(trend_data['period1_posts'])}")
        print(f"   • Period 2 posts: {len(trend_data['period2_posts'])}")
        print(f"   • API requests made: {trend_data['api_requests']}")
        print(f"   • Subreddits analyzed: {len([s for s in trend_data['subreddit_stats'] if 'error' not in trend_data['subreddit_stats'][s]])}")

        print(f"\n🚀 TOP 15 GROWING KEYWORDS (Gaining Popularity):")
        for i, (_, row) in enumerate(df.nlargest(15, 'difference').iterrows(), 1):
            print(f"   {i:2d}. {row['keyword']:30} | +{row['difference']:.3f} ({row['percent_change']:+6.1f}%)")

        print(f"\n📉 TOP 15 DECLINING KEYWORDS (Losing Popularity):")
        for i, (_, row) in enumerate(df.nsmallest(15, 'difference').iterrows(), 1):
            print(f"   {i:2d}. {row['keyword']:30} | {row['difference']:.3f} ({row['percent_change']:+6.1f}%)")

        print(f"\n🔥 MOST DISCUSSED KEYWORDS (Highest Recent Activity):")
        for i, (_, row) in enumerate(df.nlargest(15, 'period2_score').iterrows(), 1):
            print(f"   {i:2d}. {row['keyword']:30} | Score: {row['period2_score']:.3f}")

        print(f"\n📈 SUBREDDIT BREAKDOWN:")
        for subreddit, stats in trend_data['subreddit_stats'].items():
            if 'error' not in stats:
                total = stats['period1_posts'] + stats['period2_posts']
                print(f"   r/{subreddit:20} | {total:3d} posts ({stats['period1_posts']} + {stats['period2_posts']})")
            else:
                print(f"   r/{subreddit:20} | Error: {stats['error'][:50]}...")

        return df

def main():
    """Main execution function"""
    print("Reddit AI Keyword Trend Analysis")
    print("="*50)
    print("This tool analyzes AI discussion trends across major AI subreddits")
    print("comparing keyword usage from 6 months ago vs recent 3 months.\n")

    # Check if API credentials are set
    if REDDIT_CLIENT_ID == "your_client_id_here":
        print("⚠️  SETUP REQUIRED:")
        print("Please set up Reddit API credentials:")
        print("1. Go to https://www.reddit.com/prefs/apps")
        print("2. Click 'Create App' or 'Create Another App'")
        print("3. Choose 'script' type")
        print("4. Copy the client ID and secret to this notebook")
        print("5. Update REDDIT_USER_AGENT with your username")
        print("\nReddit API is free and much more generous than Twitter!")
        return None

    try:
        # Initialize analyzer
        print("🔗 Connecting to Reddit API...")
        analyzer = RedditAIAnalyzer(
            REDDIT_CLIENT_ID,
            REDDIT_CLIENT_SECRET,
            REDDIT_USER_AGENT
        )

        # Test connection
        test_subreddit = analyzer.reddit.subreddit('MachineLearning')
        print(f"✅ Connected! Testing with r/MachineLearning ({test_subreddit.subscribers:,} subscribers)")

        print(f"\n🎯 Target subreddits ({len(analyzer.ai_subreddits)}):")
        for sub in analyzer.ai_subreddits:
            print(f"   • r/{sub}")

        # Run analysis
        print(f"\n🚀 Starting analysis...")
        trend_data = analyzer.analyze_trends(months_back=6, posts_per_subreddit=50)

        # Create visualizations
        print(f"\n📊 Creating visualizations...")
        comparison_df = analyzer.create_visualizations(trend_data)

        print(f"\n✅ Analysis complete!")
        print(f"   • Processed {len(trend_data['period1_posts']) + len(trend_data['period2_posts'])} total posts")
        print(f"   • Used {trend_data['api_requests']} API requests")
        print(f"   • Analyzed {len(comparison_df)} unique keywords")

        return comparison_df, trend_data

    except Exception as e:
        print(f"❌ Error during analysis: {e}")
        print(f"   This might be due to API credentials or network connectivity.")
        return None, None

# Execute the analysis
if __name__ == "__main__":
    result_df, analysis_data = main()

# Optional: Save results to CSV
if result_df is not None:
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"reddit_ai_trends_{timestamp}.csv"
    result_df.to_csv(filename, index=False)
    print(f"\n💾 Results saved to {filename}")